CNN version 1.0
=============
Using the preprocessed Data
------------

Previously in `Preprocessing.ipynb`, we filtered various forms of inconsistencies from the data and made it appropriate for using the model.

The goal of this noteook is to run a simple CNN here.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

# Task 1: Load the data from the pickle file

In [10]:
'''Set the constant paths here'''
root = "../Data"
path = "../Data/Resized_final_data"
csv_path = "../Data/train_info.csv"
model_save_path = "../saved_models/"

Retrieve the saved data from the pickle file

In [3]:
pickle_files_names = ("Data_final.pickle",)

Formatted_Data = []


for Dfile in pickle_files_names:
    with open(os.path.join(root, Dfile), "rb") as f:
        Formatted_Data.append(pickle.load(f))

In [4]:
len(Formatted_Data)

1

In [5]:
# recollect the fragmented data

labels_mappings = Formatted_Data[0]['label_mapping']
print(labels_mappings)

train_dataSet = Formatted_Data[0]['train_data']
train_labels  = Formatted_Data[0]['train_labels']

cv_dataSet = Formatted_Data[0]['cv_data']
cv_labels = Formatted_Data[0]['cv_labels']

test_dataSet = Formatted_Data[0]['test_data']
test_labels  = Formatted_Data[0]['test_labels']

print(train_dataSet.shape, cv_dataSet.shape, test_dataSet.shape)
print(train_labels.shape, cv_labels.shape, test_labels.shape)

{'capriccio': 8, 'abstract': 0, 'history painting': 16, 'marina': 21, 'design': 12, 'quadratura': 29, 'portrait': 27, 'landscape': 19, 'symbolic painting': 35, 'allegorical painting': 2, 'miniature': 22, 'sketch and study': 33, 'panorama': 25, 'animal painting': 3, 'nude painting (nu)': 24, 'bijinga': 5, 'flower painting': 14, 'battle painting': 4, 'genre painting': 15, 'interior': 18, 'cloudscape': 11, 'mythological painting': 23, 'poster': 28, 'shan shui': 32, 'yakusha-e': 40, 'literary painting': 20, 'illustration': 17, 'caricature': 9, 'advertisement': 1, 'cityscape': 10, 'calligraphy': 7, 'religious painting': 30, 'bird-and-flower painting': 6, 'tessellation': 36, 'pastorale': 26, 'still life': 34, 'figurative': 13, 'self-portrait': 31, 'veduta': 38, 'vanitas': 37, 'wildlife painting': 39}
(17576, 100, 100, 3) (2511, 100, 100, 3) (5022, 100, 100, 3)
(17576, 41) (2511, 41) (5022, 41)


# Task 2: Spawn the CNN 

In [6]:
# The accuracy function to calculate the accuracy of the classifier

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

define the constants required hence forth:

In [7]:
batch_size = 16
patch_size = 5
depth = 16 # number of filters in a layer
num_hidden = 256

image_size = 100
num_labels = 41
num_channels = 3 # rgb images

### The CNN tensorflow graph 

In [8]:
graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  
    tf_valid_dataset = tf.constant(cv_dataSet)
    tf_test_dataset = tf.constant(test_dataSet)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, 128], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([128]))

    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, 128, 64], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[64]))
    
    layer3_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, 64, 32], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[32]))
    
    layer4_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, 32, 16], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[16]))
  
    layer5_weights = tf.Variable(tf.truncated_normal([image_size * image_size * 16, num_hidden], stddev=0.1))
    layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer6_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer6_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        
        conv = tf.nn.conv2d(hidden, layer3_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer3_biases)
        
        conv = tf.nn.conv2d(hidden, layer4_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer4_biases)
        
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        
        hidden = tf.nn.relu(tf.matmul(reshape, layer5_weights) + layer5_biases)
        return tf.matmul(hidden, layer6_weights) + layer6_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    validation_prediction = tf.nn.softmax(model(tf_valid_dataset))

In [13]:
num_steps = 501

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
  
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataSet[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
    
    saver = tf.train.Saver()
    print("Saved at: ", saver.save(session, os.path.join(model_save_path, "model_1.0/model1")))

Initialized
Minibatch loss at step 0: 520.253723
Minibatch accuracy: 0.0%
Minibatch loss at step 50: 3.528938
Minibatch accuracy: 25.0%
Minibatch loss at step 100: 3.438213
Minibatch accuracy: 6.2%
Minibatch loss at step 150: 3.207769
Minibatch accuracy: 12.5%
Minibatch loss at step 200: 3.211210
Minibatch accuracy: 25.0%
Minibatch loss at step 250: 3.372332
Minibatch accuracy: 0.0%
Minibatch loss at step 300: 2.998432
Minibatch accuracy: 25.0%
Minibatch loss at step 350: 2.908485
Minibatch accuracy: 12.5%
Minibatch loss at step 400: 3.194774
Minibatch accuracy: 12.5%
Minibatch loss at step 450: 2.650635
Minibatch accuracy: 25.0%
Minibatch loss at step 500: 2.926214
Minibatch accuracy: 0.0%
Saved at:  ../saved_models/model_1.0/model1
